In [1]:
import transformers
transformers.__version__

'4.19.2'

In [3]:
#!pip install protobuf==3.20.*

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 639.1 kB/s eta 0:00:00m eta 0:00:010:01:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.1
    Uninstalling protobuf-4.21.1:
      Successfully uninstalled protobuf-4.21.1


In [1]:
from transformers import AutoModelWithLMHead, AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "mrm8488/mbart-large-finetuned-opus-es-en-translation"#mbart-large-finetuned-bible-es-en-translation"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [2]:
TEXT =  "A lo largo de la historia de Chile han existido diversos partidos políticos, los que fueron o prohibidoso suspendidos en 1973."

In [3]:
inputs = tokenizer(
    TEXT,
    return_tensors="pt"
)
outputs = model.generate(inputs["input_ids"], max_length=40, num_beams=4, early_stopping=True)
res = tokenizer.decode(outputs[0])
print(res)

<s> Across the history of Chile, there have been several political parties, which were or banned, suspended in 1973.</s>


In [9]:
from transformers import pipeline

In [10]:
nlp = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [11]:
result = nlp(res[4:-4])

print(result)

[{'label': 'NEGATIVE', 'score': 0.9906215667724609}]


## Test con titulos de noticias


In [15]:
import pandas as pd
df = pd.read_csv("dataset-loslagos-8months.csv", 
                 sep=";", 
                 engine='python')
df.columns = ["date", "media_outlet", "url", "title", "text"]
print(len(df))
df.head()

29598


,date,media_outlet,url,title,text
0,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/reconocen-a-g...,Reconocen a guardaparques de la Región de Los ...,Distintos protagonistas de los parques naciona...
1,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/con-nuevos-ma...,Con nuevos materiales comienza plan piloto en ...,Centro de negocios Sercotec coordina acuerdos ...
2,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/centro-de-sal...,Centro de Salud Familiar CESFAM Puerto Varas i...,Las horas se solicitan en el SOME o bien a tra...
3,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/alcalde-tomas...,Alcalde Tomás Gárate presidió por primera vez ...,Los y las consejeras destacaron el hecho de vo...
4,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/galeria-de-ar...,Galería de Arte Machacoya realizará remate de ...,"Hoy viernes a las 18:30 horas, en Machacoya At..."


In [19]:
from tqdm import tqdm
sub = df[:100].copy()
sub['sentiment'] = ""

for index, row in tqdm(sub.iterrows(), desc='sub rows - sentiment', total=sub.shape[0]):
    
    inputs = tokenizer(row['title'], return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=40, num_beams=4, early_stopping=True)
    to_eng = tokenizer.decode(outputs[0])[4:-4]
    sentiment_value = nlp(to_eng)
    sub.at[index, "sentiment"] = sentiment_value


sub rows - sentiment: 100%|███████████████████████████████████████████████████████████| 100/100 [03:19<00:00,  2.00s/it]


In [25]:
pd.set_option("display.max_rows", None, "display.max_columns", None, 'display.max_colwidth', None)
sub[['title', 'sentiment']]

,title,sentiment
0,Reconocen a guardaparques de la Región de Los Lagos como actores claves en la conservación,"[{'label': 'POSITIVE', 'score': 0.9989738464355469}]"
1,Con nuevos materiales comienza plan piloto en Saltos del Petrohué,"[{'label': 'POSITIVE', 'score': 0.9602944254875183}]"
2,Centro de Salud Familiar CESFAM Puerto Varas invita a prevenir el Cáncer Cervicouterino con extensión horaria para exámenes PAP,"[{'label': 'NEGATIVE', 'score': 0.9650343060493469}]"
3,Alcalde Tomás Gárate presidió por primera vez la octava sesión del Consejo Comunal de la Sociedad Civil COSOC Puerto Varas,"[{'label': 'POSITIVE', 'score': 0.9859645366668701}]"
4,Galería de Arte Machacoya realizará remate de obras de artistas de la zona,"[{'label': 'POSITIVE', 'score': 0.9991785883903503}]"
5,Municipio llamará a licitación construcción de pasarela peatonal en Los Notros,"[{'label': 'NEGATIVE', 'score': 0.9589917659759521}]"
6,3era Mesa de Reactivación Económica circunscribe su radio de acción,"[{'label': 'NEGATIVE', 'score': 0.9504976868629456}]"
7,Gremio médico rechaza cierre de camas críticas implementadas en Hospital Ancud,"[{'label': 'NEGATIVE', 'score': 0.9135583639144897}]"
8,Asistentes de educación municipal en Osorno verán incrementadas sus remuneraciones,"[{'label': 'POSITIVE', 'score': 0.9925214648246765}]"
9,CONADI llama a renovar directivas de comunidades indígenas,"[{'label': 'NEGATIVE', 'score': 0.7058996558189392}]"
